In [76]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import csv
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [77]:
# Lectura de datos
nodes = pd.read_csv("../datos/musae_git_target.csv")
nodes = nodes.sort_values(by='id_node')
atributeDegreeCentrality = pd.read_csv("../metricas/degree_centrality.csv")
atributeDegreeCentrality = atributeDegreeCentrality.sort_values(by="id_node")
atributeClosenessCentrality = pd.read_csv("../metricas/closeness_centrality.csv")
atributeClosenessCentrality = atributeClosenessCentrality.sort_values(by='id_node')
atributeBetweennessCentrality = pd.read_csv("../metricas/betweenness_centrality.csv")
atributeBetweennessCentrality = atributeBetweennessCentrality.sort_values(by='id_node')
atributeClusteringCoefficient = pd.read_csv("../metricas/clustering_coefficient.csv")
atributeClusteringCoefficient = atributeClusteringCoefficient.sort_values(by='id_node')
atributeGeneralizedDegree = pd.read_csv("../metricas/generalized_degree.csv")
atributeGeneralizedDegree = atributeGeneralizedDegree.sort_values(by='id_node')
atributeTriangles = pd.read_csv("../metricas/triangles.csv")
atributeTriangles = atributeTriangles.sort_values(by='id_node')
atributeGreedyModularityCommunities = pd.read_csv("../metricas/greedy_modularity_communities.csv")
atributeGreedyModularityCommunities = atributeGreedyModularityCommunities.sort_values(by='id_node')
atributeGreedyModularityCommunities = atributeGreedyModularityCommunities.iloc[:, ::-1]
atributeCoreNumber = pd.read_csv("../metricas/core_number.csv")
atributeCoreNumber = atributeCoreNumber.sort_values(by='id_node')
atributeAsynLpaCommunities = pd.read_csv("../metricas/asyn_lpa_communities.csv")
atributeAsynLpaCommunities = atributeAsynLpaCommunities.sort_values(by='id_node')


In [78]:
# Vamos a meter todos los atributos en la tabla 

# Renombrar columnas
atributeDegreeCentrality.rename(columns={'Degree Centrality': 'degree_centrality'}, inplace=True)
atributeClosenessCentrality.rename(columns={'Closeness Centrality': 'closeness_centrality'}, inplace=True)
atributeBetweennessCentrality.rename(columns={'Betweenness Centrality': 'betweenness_centrality'}, inplace=True)
atributeClusteringCoefficient.rename(columns={'Clustering Coefficient': 'clustering_coefficient'}, inplace=True)
atributeGeneralizedDegree.rename(columns={'Generalized Degree': 'generalized_degree'}, inplace=True)
atributeTriangles.rename(columns={'Triangles': 'triangles'}, inplace=True)
atributeGreedyModularityCommunities.rename(columns={'Community': 'greedy_modularity_communities'}, inplace=True)
atributeCoreNumber.rename(columns={'Core Number': 'core_number'}, inplace=True)
atributeAsynLpaCommunities.rename(columns={'Community': 'asyn_lpa_communities'}, inplace=True)

# Convertir la columna 'id_node' a tipo de datos str en todos los DataFrames
nodes['id_node'] = nodes['id_node'].astype(str)
atributeDegreeCentrality['id_node'] = atributeDegreeCentrality['id_node'].astype(str)
atributeClosenessCentrality['id_node'] = atributeClosenessCentrality['id_node'].astype(str)
atributeBetweennessCentrality['id_node'] = atributeBetweennessCentrality['id_node'].astype(str)
atributeClusteringCoefficient['id_node'] = atributeClusteringCoefficient['id_node'].astype(str)
atributeGeneralizedDegree['id_node'] = atributeGeneralizedDegree['id_node'].astype(str)
atributeTriangles['id_node'] = atributeTriangles['id_node'].astype(str)
atributeGreedyModularityCommunities['id_node'] = atributeGreedyModularityCommunities['id_node'].astype(str)
atributeCoreNumber['id_node'] = atributeCoreNumber['id_node'].astype(str)
atributeAsynLpaCommunities['id_node'] = atributeAsynLpaCommunities['id_node'].astype(str)

# Fusionar todos los DataFrames en uno
nodes = pd.merge(nodes, atributeDegreeCentrality, on='id_node', how='outer')
nodes = pd.merge(nodes, atributeClosenessCentrality, on='id_node', how='outer')
nodes = pd.merge(nodes, atributeBetweennessCentrality, on='id_node', how='outer')
nodes = pd.merge(nodes, atributeClusteringCoefficient, on='id_node', how='outer')
nodes = pd.merge(nodes, atributeGeneralizedDegree, on='id_node', how='outer')
nodes = pd.merge(nodes, atributeTriangles, on='id_node', how='outer')
nodes = pd.merge(nodes, atributeGreedyModularityCommunities, on='id_node', how='outer')
nodes = pd.merge(nodes, atributeCoreNumber, on='id_node', how='outer')
nodes = pd.merge(nodes, atributeAsynLpaCommunities, on='id_node', how='outer')

# Convertir 'id_node' a tipo de datos numérico, manejando errores
nodes['id_node'] = pd.to_numeric(nodes['id_node'], errors='coerce')

# Eliminar filas con id_node NaN (resultantes de conversiones fallidas)
nodes = nodes.dropna(subset=['id_node'])

# Convertir 'id_node' a tipo entero
nodes['id_node'] = nodes['id_node'].astype(int)

# Ordenar por id_node
nodes = nodes.sort_values(by='id_node', ascending=True)

# Convertir 'id_node' de nuevo a string si es necesario
nodes['id_node'] = nodes['id_node'].astype(str)

nodes.head(7)


,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,generalized_degree,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,Counter({0: 1}),0.0,0,1.0,3.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,"Counter({0: 3, 1: 2, 3: 2, 2: 1})",5.0,1,6.0,3.0
11112,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,Counter({0: 1}),0.0,0,1.0,3.0
22223,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,Counter({0: 5}),0.0,2,4.0,3.0
31034,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,Counter({0: 2}),0.0,5,2.0,3.0
32145,5,j6montoya,0.0,0.000027,0.343412,0.000000e+00,0.000000,Counter({0: 1}),0.0,0,1.0,3.0
33256,6,sfate,0.0,0.000159,0.372244,2.098552e-06,0.333333,"Counter({2: 2, 1: 2, 0: 1, 4: 1})",5.0,0,6.0,3.0


In [79]:
# Obtener la fila correspondiente al ID_NODE específico (por ejemplo, ID = 0)
id_especifico = '29982'  # Convertir el ID_NODE a tipo de datos str
fila_id_especifico = nodes[nodes['id_node'] == id_especifico]

fila_id_especifico.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,generalized_degree,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
22204,29982,dead-horse,0.0,0.01297,0.403407,0.001086,0.035335,"Counter({3: 37, 5: 36, 8: 33, 6: 32, 2: 30, 4:...",4216.0,1,34.0,3.0


In [80]:
# Discretización de la columna generalized_degree

In [81]:
#Normalizar columnas
columns_to_normalize = ['clustering_coefficient', 'triangles', 'asyn_lpa_communities','greedy_modularity_communities', 'Core number']  # Reemplaza con los nombres de las columnas que deseas normalizar

# Inicializa el escalador
scaler = MinMaxScaler()

# Aplica el escalador a las columnas seleccionadas
nodes[columns_to_normalize] = scaler.fit_transform(nodes[columns_to_normalize])
nodes.head()

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,generalized_degree,triangles,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,Counter({0: 1}),0.000000,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,"Counter({0: 3, 1: 2, 3: 2, 2: 1})",0.000062,0.002227,0.151515,0.0
11112,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,Counter({0: 1}),0.000000,0.000000,0.000000,0.0
22223,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,Counter({0: 5}),0.000000,0.004454,0.090909,0.0
31034,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,Counter({0: 2}),0.000000,0.011136,0.030303,0.0


In [82]:
#Tablas con todos los atributos sin los de agrupamiento

nodesWithoutClustering = nodes.copy()

# Eliminar las columnas de agrupamiento
nodesWithoutClustering = nodesWithoutClustering.drop(columns=['clustering_coefficient', 'generalized_degree','triangles'])

nodesWithoutClustering.head(7)


,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,greedy_modularity_communities,Core number,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.002227,0.151515,0.0
11112,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,0.000000,0.0
22223,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.004454,0.090909,0.0
31034,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.011136,0.030303,0.0
32145,5,j6montoya,0.0,0.000027,0.343412,0.000000e+00,0.000000,0.000000,0.0
33256,6,sfate,0.0,0.000159,0.372244,2.098552e-06,0.000000,0.151515,0.0


In [83]:
#Tablas con todos los atributos sin los de comunidades

nodesWithoutCommunity = nodes.copy()

# Eliminar las columnas de agrupamiento
nodesWithoutCommunity = nodesWithoutCommunity.drop(columns=['greedy_modularity_communities','asyn_lpa_communities'])

nodesWithoutCommunity.head(7)

,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,generalized_degree,triangles,Core number
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,Counter({0: 1}),0.000000,0.000000
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,"Counter({0: 3, 1: 2, 3: 2, 2: 1})",0.000062,0.151515
11112,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,Counter({0: 1}),0.000000,0.000000
22223,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,Counter({0: 5}),0.000000,0.090909
31034,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,Counter({0: 2}),0.000000,0.030303
32145,5,j6montoya,0.0,0.000027,0.343412,0.000000e+00,0.000000,Counter({0: 1}),0.000000,0.000000
33256,6,sfate,0.0,0.000159,0.372244,2.098552e-06,0.333333,"Counter({2: 2, 1: 2, 0: 1, 4: 1})",0.000062,0.151515


In [84]:
#Tablas con todos los atributos sin los de nucleos

nodesWithoutKernel = nodes.copy()

# Eliminar las columnas de agrupamiento
nodesWithoutKernel = nodesWithoutKernel.drop(columns=['Core number'])

nodesWithoutKernel.head(7)


,id_node,name,ml_target,degree_centrality,closeness_centrality,betweenness_centrality,clustering_coefficient,generalized_degree,triangles,greedy_modularity_communities,asyn_lpa_communities
0,0,Eiryyy,0.0,0.000027,0.275005,0.000000e+00,0.000000,Counter({0: 1}),0.000000,0.000000,0.0
1,1,shawflying,0.0,0.000212,0.294956,1.149733e-06,0.178571,"Counter({0: 3, 1: 2, 3: 2, 2: 1})",0.000062,0.002227,0.0
11112,2,JpMCarrilho,1.0,0.000027,0.261845,0.000000e+00,0.000000,Counter({0: 1}),0.000000,0.000000,0.0
22223,3,SuhwanCha,0.0,0.000133,0.278718,5.316292e-05,0.000000,Counter({0: 5}),0.000000,0.004454,0.0
31034,4,sunilangadi2,1.0,0.000053,0.243084,6.134318e-09,0.000000,Counter({0: 2}),0.000000,0.011136,0.0
32145,5,j6montoya,0.0,0.000027,0.343412,0.000000e+00,0.000000,Counter({0: 1}),0.000000,0.000000,0.0
33256,6,sfate,0.0,0.000159,0.372244,2.098552e-06,0.333333,"Counter({2: 2, 1: 2, 0: 1, 4: 1})",0.000062,0.000000,0.0
